# Tekstclassificatietaak

Zoals we hebben vermeld, richten we ons op een eenvoudige tekstclassificatietaak gebaseerd op het **AG_NEWS**-dataset, waarbij nieuwsheadlines worden geclassificeerd in een van de 4 categorieën: Wereld, Sport, Zakelijk en Wetenschap/Technologie.

## Het Dataset

Dit dataset is ingebouwd in de [`torchtext`](https://github.com/pytorch/text) module, waardoor we er gemakkelijk toegang toe hebben.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Hier bevatten `train_dataset` en `test_dataset` collecties die respectievelijk paren van label (nummer van klasse) en tekst retourneren, bijvoorbeeld:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Dus, laten we de eerste 10 nieuwe koppen uit onze dataset afdrukken:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Omdat datasets iterators zijn, moeten we de gegevens omzetten naar een lijst als we deze meerdere keren willen gebruiken:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisatie

Nu moeten we tekst omzetten in **nummers** die kunnen worden weergegeven als tensors. Als we een representatie op woordniveau willen, moeten we twee dingen doen:
* een **tokenizer** gebruiken om tekst op te splitsen in **tokens**
* een **vocabulaire** opbouwen van die tokens.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Met behulp van vocabulaire kunnen we onze getokeniseerde string eenvoudig coderen in een reeks cijfers:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Zak met Woorden tekstrepresentatie

Omdat woorden betekenis vertegenwoordigen, kunnen we soms de betekenis van een tekst achterhalen door alleen naar de afzonderlijke woorden te kijken, ongeacht hun volgorde in de zin. Bijvoorbeeld, bij het classificeren van nieuws, zullen woorden zoals *weer*, *sneeuw* waarschijnlijk wijzen op *weersvoorspelling*, terwijl woorden zoals *aandelen*, *dollar* eerder zouden duiden op *financieel nieuws*.

**Zak met Woorden** (BoW) vectorrepresentatie is de meest gebruikte traditionele vectorrepresentatie. Elk woord is gekoppeld aan een vectorindex, en het vectorelement bevat het aantal keren dat een woord voorkomt in een bepaald document.

![Afbeelding die laat zien hoe een zak met woorden vectorrepresentatie in het geheugen wordt weergegeven.](../../../../../translated_images/nl/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Je kunt BoW ook zien als de som van alle one-hot-gecodeerde vectoren voor individuele woorden in de tekst.

Hieronder staat een voorbeeld van hoe je een zak met woorden representatie kunt genereren met behulp van de Scikit Learn python bibliotheek:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Om de bag-of-words vector te berekenen uit de vectorrepresentatie van onze AG_NEWS dataset, kunnen we de volgende functie gebruiken:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Opmerking:** Hier gebruiken we de globale variabele `vocab_size` om de standaardgrootte van de woordenschat op te geven. Aangezien de woordenschat vaak behoorlijk groot is, kunnen we de grootte van de woordenschat beperken tot de meest voorkomende woorden. Probeer de waarde van `vocab_size` te verlagen en de onderstaande code uit te voeren, en kijk hoe dit de nauwkeurigheid beïnvloedt. Je kunt een lichte daling in nauwkeurigheid verwachten, maar niet dramatisch, in ruil voor betere prestaties.


## Een BoW-classificator trainen

Nu we hebben geleerd hoe we een Bag-of-Words-representatie van onze tekst kunnen maken, laten we een classificator hierop trainen. Eerst moeten we onze dataset zo omzetten voor training, dat alle positionele vectorrepresentaties worden omgezet naar een bag-of-words-representatie. Dit kan worden bereikt door de functie `bowify` door te geven als de parameter `collate_fn` aan de standaard torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Laten we nu een eenvoudige classifier-neuraal netwerk definiëren dat één lineaire laag bevat. De grootte van de invoervector is gelijk aan `vocab_size`, en de uitvoergrootte komt overeen met het aantal klassen (4). Omdat we een classificatietaak oplossen, is de uiteindelijke activatiefunctie `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Nu zullen we de standaard PyTorch-trainingslus definiëren. Omdat onze dataset vrij groot is, zullen we voor ons onderwijsdoel slechts één epoch trainen, en soms zelfs minder dan een epoch (het specificeren van de parameter `epoch_size` stelt ons in staat om de training te beperken). We zouden ook de geaccumuleerde trainingsnauwkeurigheid tijdens de training rapporteren; de frequentie van rapportage wordt gespecificeerd met de parameter `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrammen, TriGrammen en N-Grammen

Een beperking van de bag-of-words aanpak is dat sommige woorden deel uitmaken van meerwoorduitdrukkingen. Bijvoorbeeld, het woord 'hot dog' heeft een compleet andere betekenis dan de woorden 'hot' en 'dog' in andere contexten. Als we de woorden 'hot' en 'dog' altijd met dezelfde vectoren representeren, kan dat ons model verwarren.

Om dit probleem aan te pakken, worden **N-gram representaties** vaak gebruikt bij methoden voor documentclassificatie, waarbij de frequentie van elk woord, bi-woord of tri-woord een nuttige eigenschap is voor het trainen van classifiers. In een bigram-representatie voegen we bijvoorbeeld alle woordparen toe aan de vocabulaire, naast de oorspronkelijke woorden.

Hieronder staat een voorbeeld van hoe je een bigram bag-of-words representatie kunt genereren met behulp van Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Het grootste nadeel van de N-gram aanpak is dat de omvang van de woordenschat extreem snel begint te groeien. In de praktijk moeten we de N-gram representatie combineren met enkele technieken voor dimensiereductie, zoals *embeddings*, die we in de volgende eenheid zullen bespreken.

Om de N-gram representatie te gebruiken in ons **AG News** dataset, moeten we een speciale ngram-woordenschat opbouwen:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


We zouden dezelfde code als hierboven kunnen gebruiken om de classifier te trainen, maar dat zou erg geheugeninefficiënt zijn. In de volgende eenheid zullen we een bigram-classifier trainen met behulp van embeddings.

> **Opmerking:** Je kunt alleen die ngrams behouden die vaker in de tekst voorkomen dan een opgegeven aantal keren. Dit zorgt ervoor dat zeldzame bigrams worden weggelaten en de dimensie aanzienlijk wordt verkleind. Stel hiervoor de parameter `min_freq` in op een hogere waarde en observeer hoe de lengte van de woordenschat verandert.


## Termfrequentie Inverse Documentfrequentie TF-IDF

In de BoW-representatie worden woordvoorkomens gelijk gewogen, ongeacht het woord zelf. Het is echter duidelijk dat frequente woorden, zoals *een*, *in*, enz., veel minder belangrijk zijn voor de classificatie dan gespecialiseerde termen. In feite zijn bij de meeste NLP-taken sommige woorden relevanter dan andere.

**TF-IDF** staat voor **termfrequentie–inverse documentfrequentie**. Het is een variatie op bag of words, waarbij in plaats van een binaire 0/1-waarde die aangeeft of een woord in een document voorkomt, een drijvende-kommawaarde wordt gebruikt die gerelateerd is aan de frequentie van woordvoorkomen in de corpus.

Meer formeel wordt het gewicht $w_{ij}$ van een woord $i$ in document $j$ gedefinieerd als:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
waarbij
* $tf_{ij}$ het aantal voorkomens van $i$ in $j$ is, oftewel de BoW-waarde die we eerder hebben gezien
* $N$ het aantal documenten in de collectie is
* $df_i$ het aantal documenten is waarin het woord $i$ voorkomt in de hele collectie

De TF-IDF-waarde $w_{ij}$ neemt proportioneel toe met het aantal keren dat een woord in een document voorkomt en wordt gecorrigeerd door het aantal documenten in de corpus waarin het woord voorkomt. Dit helpt om te compenseren voor het feit dat sommige woorden vaker voorkomen dan andere. Bijvoorbeeld, als het woord in *elke* document in de collectie voorkomt, dan geldt $df_i=N$, en $w_{ij}=0$, en die termen worden volledig genegeerd.

Je kunt eenvoudig een TF-IDF-vectorisatie van tekst maken met behulp van Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Conclusie

Hoewel TF-IDF-representaties frequentiegewicht toekennen aan verschillende woorden, zijn ze niet in staat om betekenis of volgorde weer te geven. Zoals de beroemde taalkundige J. R. Firth in 1935 zei: "De volledige betekenis van een woord is altijd contextueel, en geen enkele studie van betekenis los van context kan serieus worden genomen." Later in de cursus zullen we leren hoe we contextuele informatie uit tekst kunnen vastleggen met behulp van taalmodellen.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
